In [10]:
import pandas as pd

# Function to import the data from the Excel file
def load_datasets(filename='benchmark_target.xlsx'):
    df_benchmark = pd.read_excel(filename, sheet_name='Benchmark')
    df_target = pd.read_excel(filename, sheet_name='Target')
    return df_benchmark, df_target

# Function to calculate the necessary metrics for both datasets
def calculate_metrics(df, target_size, weights):
    # Initialize metrics with default values
    metrics = {
        'Debt': None,
        'DPD': None,
        'GR': None,
        'LP': None,
        'GRDPD': None,
        'GRLP': None,
        'collection': None,
        'payments': None,
    }

    # Calculate averages for the available columns
    for col in ['Debt', 'DPD', 'GR', 'LP', 'GRDPD', 'GRLP']:
        if col in df.columns:
            metrics[col] = df[col].mean()

    # Check for collection and payments
    if 'Payments' in df.columns:
        # If Payments column exists
        total_paid = df['Payments'].sum()
        total_debt = df['Debt'].sum() if 'Debt' in df.columns else 0
        metrics['collection'] = total_paid / (total_debt + total_paid) if total_debt + total_paid != 0 else 0
        metrics['payments'] = df['Payments'].notnull().sum() / len(df)
    elif 'LP' in df.columns:
        # If LP column exists
        metrics['payments'] = df['LP'].notnull().sum() / len(df)
    else:
        # If neither Payments nor LP columns exist
        metrics['collection'] = None
        metrics['payments'] = None

    return metrics

# Objective function to minimize the difference between benchmark and target metrics
def objective_function(x, df_benchmark, df_target, target_metrics, weights):
    selected_indices = [int(i) for i in x]  # Convert row numbers to integers
    sample = df_benchmark.iloc[selected_indices]  # Select rows based on indices
    sample_metrics = calculate_metrics(sample, len(selected_indices), weights)  # Calculate metrics for sample

    # Calculate the weighted percentage difference
    percentage_diff = 0
    for key in weights.keys():
        percentage_diff += weights[key] * abs((sample_metrics[key] - target_metrics[key]) / target_metrics[key]) * 100

    return percentage_diff

# Greedy Sampling function to ensure uniqueness
def greedy_sampling(df_benchmark, df_target, weights, target_size):
    target_metrics = calculate_metrics(df_target, target_size, weights)
    selected_indices = set()  # Using a set to keep track of unique indices

    # Greedy approach: iteratively select the best row minimizing the difference
    while len(selected_indices) < target_size:
        best_index = None
        best_diff = float('inf')

        for i in range(len(df_benchmark)):
            if i in selected_indices:
                continue  # Skip if already selected

            temp_indices = selected_indices.union({i})
            temp_sample = df_benchmark.iloc[list(temp_indices)]
            temp_metrics = calculate_metrics(temp_sample, len(temp_indices), weights)
            temp_diff = sum(weights[key] * abs((temp_metrics[key] - target_metrics[key]) / target_metrics[key]) * 100
                            for key in weights)

            if temp_diff < best_diff:
                best_diff = temp_diff
                best_index = i

        if best_index is not None:
            selected_indices.add(best_index)

    return list(selected_indices)

# Function to write results to Excel, updated to take both df_benchmark and df_target
def write_results_to_excel(benchmark_df, target_df, selected_indices, filename='benchmark_target.xlsx'):
    with pd.ExcelWriter(filename, mode='a', if_sheet_exists='replace') as writer:
        new_sample_col = f'Sample {len(benchmark_df.columns) - 1}'  # For a new sample column

        # Add a new column for selected rows
        benchmark_df[new_sample_col] = 0
        benchmark_df.loc[selected_indices, new_sample_col] = 1  # Set to 1 for selected rows

        # Write both DataFrames to separate sheets in the same Excel file
        benchmark_df.to_excel(writer, sheet_name='Benchmark', index=False)
        target_df.to_excel(writer, sheet_name='Target', index=False)

# Main function to run the process
def main():
    df_benchmark, df_target = load_datasets('benchmark_target.xlsx')  # Load datasets from Excel

    # Define the column weights
    weights = {
        'Deb': 0.2,
        'GR': 0.2,
        'DPD': 0.2,
        'LP': 0.2,
        'collection': 0.1,
        'payments': 0.1
    }

    # Perform greedy sampling to select the best rows
    
    target_size = 95  # Adjust the target size as needed
    selected_indices = greedy_sampling(df_benchmark, df_target, weights, target_size)

    # Write the results back to the Excel file, adding a new sample column
    write_results_to_excel(df_benchmark, df_target, selected_indices)

# Run the main function
if __name__ == "__main__":
    main()
